<a href="https://colab.research.google.com/github/kangmyoungseok/RugPullTimeStamp/blob/main/RugPullTimeStamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Main py

In [22]:
#RugPull이 일어난 시점의 TimeStamp를 구해서 Paris_v1.x파일에 추가한다.
#Pre-Condition : Pairs_v1.4.csv파일
import json
import pandas as pd
from pandas.core.frame import DataFrame
import datetime
# json 파일과 Pair를 메모리에 먼저 올린다.
with open('/content/drive/MyDrive/swap.json', 'r') as f:
    swap_data = json.load(f)

with open('/content/drive/MyDrive/mint.json', 'r') as f:
    mint_data = json.load(f)

with open('/content/drive/MyDrive/burn.json', 'r') as f:
    burn_data = json.load(f)

#메모리 공간 활용을 위해서 Pairs파일에서 필요한 열(id,token0,token1)만 올리기
#아래의 데이터는 swap/mint/burn에서 amount0이 이더인지 amount1이 이더인지를 판단하기 위해서 필요
datas = pd.read_csv('/content/drive/MyDrive/Pairs_v1.5.csv',usecols=['id','token0.name','token1.name']).to_dict('records')
token_pairs = {}
for data in datas:
    tmp_dict = {data['id'] : {'token0.name':data['token0.name'],'token1.name':data['token1.name']}}
    token_pairs.update(tmp_dict)
del datas


# Pair_address를 주면 토큰이 token0인지 token1인지 판단. 0,1로 리턴
def token_index(pair_address): 
    if(token_pairs[pair_address]['token0.name'] =='Wrapped Ether' ):
      return  1
    else:
      return 0
  
#get_xxx_mean_period 는 mint/burn/swap의 ActivePeriod 상에서의 비율 표시
def get_mint_mean_period(mint_data_transaction,initial_timestamp):
    count = len(mint_data_transaction)
    if(count == 0):
      return 0
    mint_time_add = 0
    for transaction in mint_data_transaction:
      mint_time_add = mint_time_add + int(transaction['timestamp']) - initial_timestamp
    return mint_time_add / count

def get_swap_mean_period(swap_data_transaction,initial_timestamp):
    count = len(swap_data_transaction)
    if(count == 0):
      return 0
    swap_time_add = 0
    for transaction in swap_data_transaction:
      swap_time_add = swap_time_add +  int(transaction['timestamp']) - initial_timestamp
    return swap_time_add / count


def get_burn_mean_period(burn_data_transaction,initial_timestamp):
    count = len(burn_data_transaction)
    if(count == 0):
      return 0
    burn_time_add = 0
    for transaction in burn_data_transaction:
      burn_time_add = burn_time_add + int(transaction['timestamp']) - initial_timestamp
    return burn_time_add / count

def swap_IO_rate(swap_data_transaction,index):
  swapIn = 0
  swapOut = 0
  if(index == 1): #amount0이 이더.
    for data in swap_data_transaction:
      if(data['amount0In'] == '0'): #amount0Out이 0이 아니란 말. 
        swapOut = swapOut + 1
      else:   
        swapIn = swapIn + 1
  else:         #amount1이 이더
    for data in swap_data_transaction:
      if(data['amount1In'] == '0'):
        swapOut = swapOut + 1
      else:
        swapIn = swapIn +1
  
  return swapIn,swapOut 

def get_last_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction):
  #mint_data_transaction은 0일 수가 없다. 
  swap_len = len(swap_data_transaction)
  burn_len = len(burn_data_transaction)
  #Case 1 Swap / Burn 전부 0 인경우
  if(swap_len == 0 and burn_len == 0):
    return int(mint_data_transaction[-1]['timestamp'])
  #Case 2 Swap_transaction이 0 인경우
  if(swap_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp']))
  #Case 3 Burn Transaction이 0 인경우
  if(burn_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))
  #Case 4 전부다 있는 경우
  return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))
  

def get_swap_amount(swap_data_transaction,j,eth_amountIn,eth_amountOut):
  if(swap_data_transaction[j][eth_amountIn] == '0'): #amountIn이 0 이면 out이라는 거지.
    return float(swap_data_transaction[j][eth_amountOut]) * (-1)
  else:
    return float(swap_data_transaction[j][eth_amountIn])


def get_timestamp(data_transaction,index):
  try:
    return data_transaction[index]['timestamp']
  except:
    return '99999999999'


def check_rugpull(initial_Eth, current_Eth):
  if ( float(current_Eth) / float(initial_Eth) < 0.2 ):
    print('rugpull occur')
    return True
  else:
    return False

def get_rugpull_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction,index):
    if(index == 1):
      eth_amount = 'amount0'
      eth_amountIn = 'amount0In'
      eth_amountOut = 'amount0Out'
    else:
      eth_amount = 'amount1'
      eth_amountIn = 'amount1In'
      eth_amountOut = 'amount1Out'
    
    #각각 배열의 길이를 구해서 반복문 끝 조절
    mint_count = len(mint_data_transaction)
    swap_count = len(swap_data_transaction)
    burn_count = len(burn_data_transaction)

    
    #유동성에 남아있는 이더를 지속적으로 보면서 러그풀이 언제 발생하는지 탐지
    initial_Liquidity_Eth = float(mint_data_transaction[0][eth_amount])
    current_Liquidity_Eth = 0
    rugpull_timestamp = 0

    i,j,k = 0,0,0   #mint,swap,burn 배열의 인덱스
    while True:
        
      next_timestamp = min(get_timestamp(mint_data_transaction,i),get_timestamp(burn_data_transaction,k))

      #swap 인 경우 current_Eth 더하는 로직 / 러그풀 타임스탬프 체크
      while(get_timestamp(swap_data_transaction,j) < next_timestamp ):
        #swap이 제일 타임스탬프가 작은 경우니까 스왑에 맞게 amount +-를 하면 된다.
        current_Liquidity_Eth = current_Liquidity_Eth + get_swap_amount(swap_data_transaction,j,eth_amountIn,eth_amountOut)
        if(check_rugpull(initial_Liquidity_Eth,current_Liquidity_Eth)):
          return get_timestamp(swap_data_transaction,j), float(current_Liquidity_Eth / initial_Liquidity_Eth) -1, True
        j = j+1

      #mint 인 경우 curruent_Eth 더하는 로직
      if(next_timestamp == get_timestamp(mint_data_transaction,i)): #mint가 최소라면, + 한다.
        if(next_timestamp == '99999999999'):  #이건 rugpull이 없는 경우
          try:
            return max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']), float(current_Liquidity_Eth / initial_Liquidity_Eth), False
          except:
            return '-1',-1
        current_Liquidity_Eth = current_Liquidity_Eth + float(mint_data_transaction[i][eth_amount]) 
        i = i+1

      #burn 인 경우 current_Eth 빼는 로직 / 러그풀 타임스탬프 체
      else:
        print(burn_data_transaction)
        current_Liquidity_Eth = current_Liquidity_Eth - float(burn_data_transaction[k][eth_amount])
        if(check_rugpull(initial_Liquidity_Eth,current_Liquidity_Eth)):
          return get_timestamp(burn_data_transaction,k), float(current_Liquidity_Eth / initial_Liquidity_Eth) -1, True
        k = k+1

total_list = []

for pair_address,swap_list in mint_data.items():
    tmp_dict = {}
    mint_data_transaction = mint_data[pair_address]
    swap_data_transaction = swap_data[pair_address]
    burn_data_transaction = burn_data[pair_address]
    # 각각의 count 구하기
    mint_count = len(mint_data_transaction)
    swap_count = len(swap_data_transaction)
    burn_count = len(burn_data_transaction)

    # Mint/Burn/Swap의 Active Period 상의 분포 
    initial_timestamp = int(mint_data_transaction[0]['timestamp'])
    last_timestamp = get_last_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction)
    active_period = last_timestamp - initial_timestamp
    mint_mean_period = int(get_mint_mean_period(mint_data_transaction,initial_timestamp))
    swap_mean_period = int(get_swap_mean_period(swap_data_transaction,initial_timestamp))
    burn_mean_period = int(get_burn_mean_period(burn_data_transaction,initial_timestamp))

    #SwapIn/SwapOut 비율
    swapIn,swapOut = swap_IO_rate(swap_data_transaction,token_index(pair_address))    

    #rugpull timestamp , 러그풀 시점에서 유동성 풀에 있는 이더 변화량(rugpull timestamp / change)
    rugpull_timestamp, rugpull_change, is_rugpull = get_rugpull_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction,token_index(pair_address))

    #rugpull 이 시작부터 끝날때까지 경과한 시간
    rugpull_proceeding_time = int(rugpull_timestamp) - int(initial_timestamp)

    #데이터 모아서 하나의 dictionary로 
    tmp_dict['id'] = pair_address
    tmp_dict['mint_count'] = mint_count
    tmp_dict['swap_count'] = swap_count
    tmp_dict['burn_count'] = burn_count
    tmp_dict['mint_mean_period'] = mint_mean_period
    tmp_dict['swap_mean_period'] = swap_mean_period
    tmp_dict['burn_mean_period'] = burn_mean_period
    tmp_dict['swapIn'] = swapIn
    tmp_dict['swapOut'] = swapOut
    tmp_dict['active_period'] = active_period
    tmp_dict['rugpull_timestamp'] = rugpull_timestamp
    tmp_dict['rugpull_timestamp_date'] = datetime.datetime.fromtimestamp(int(rugpull_timestamp)).strftime('%Y-%m-%d %H:%M:%S')
    tmp_dict['rugpull_change'] = rugpull_change 
    tmp_dict['rugpull_proceeding_time'] = rugpull_proceeding_time / 86400
    tmp_dict['is_rugpull'] = is_rugpull
    #전체 배열에 하나의 원소로 더한다.
    total_list.append(tmp_dict)

DataFrame(total_list).to_csv('result.csv',encoding='utf-8-sig')

KeyError: ignored

#SwapIn/Out 구현

In [ ]:
def swap_IO_rate(swap_data_transaction,index):
  swapIn = 0
  swapOut = 0
  if(index == 1): #amount0이 이더.
    for data in swap_data_transaction:
      if(data['amount0In'] == '0'): #amount0Out이 0이 아니란 말. 
        swapOut = swapOut + 1
      else:   
        swapIn = swapIn + 1
  else:         #amount1이 이더
    for data in swap_data_transaction:
      if(data['amount1In'] == '0'):
        swapOut = swapOut + 1
      else:
        swapIn = swapIn +1
  
  return swapIn,swapOut 


#get_rugpull_timestamp 구현

In [ ]:
def get_rugpull_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction,index):
    if(index == 1):
      amount = 'amount0'
      amountIn = 'amount0In'
      amountOut = 'amount0Out'
    else:
      amount = 'amount1'
      amountIn = 'amount1In'
      amountOut = 'amount1Out'
    
    #각각 배열의 길이를 구해서 반복문 끝 조절
    mint_count = len(mint_data_transaction)
    swap_count = len(swap_data_transaction)
    burn_count = len(burn_data_transaction)

    
    #유동성에 남아있는 이더를 지속적으로 보면서 러그풀이 언제 발생하는지 탐지
    initial_Liquidity_Eth = mint_data_transaction[0][amount]
    current_Liquidity_Eth = 0
    rugpull_timestamp = 0

    while True:
      i,j,k = 0,0,0   #mint,swap,burn 배열의 인덱스

      if(mint_data_transaction[i]['timestamp'] <= burn_data_transaction[k]['timestamp']): #mint가 더 작인경우
        


    #CASE1 : Swap/Burn이 하나도 없는 경우
    if ((swap_count == 0) and (burn_count == 0):# swap/burn이 하나도 없는 경우, 러그풀일 수가 없네 가장 마지막 timestamp 찍고 나가라
      return mint_data_transaction[-1]['timestamp'],'no rug pull'

    #CASE2 : Swap이 하나도 없는 경우,mint/burn만 비교한다.
    if (swap_count == 0):
      i,j = 0,0
      while(True):
        if(mint_data_transaction[i]['timestamp'] <= burn_data_transaction[j]['timestamp']):
          current_Liquidity_Eth = current_Liquidity_Eth + mint_data_transaction[i][amount]
          i = i+1
          if(i == mint_count):  #mint데이터가 더이상 없으면 burn 데이터 전부 빼면서 조건 체크한다.
            for j in (j,burn_count):
              
            
              

        



    initial_Liquidity_Eth = mint_data_transaction[0]
    
    if(swap_count == 0):

    while(1):
      i,j,k = 0,0,0
      max_timestamp = 


  return rugpull_timestamp

IndentationError: ignored

In [ ]:
def get_timestamp(data_transaction,index):
  try:
    return data_transaction[index]['timestamp']
  except:
    return '999999999999'

1


In [ ]:
def check_rugpull(initial_Eth, current_Eth):
  if (initial_Eth/current_Eth)

2
3
4
5
6
7
8
9


In [ ]:
blank_list = []
blank_list[0]

IndexError: ignored

In [ ]:
mint_data_transaction

[{'amount0': '25000000000',
  'amount1': '11',
  'sender': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d',
  'test': '123',
  'timestamp': '1626033500',
  'to': '0x164b574f9922464092103d812f94b731ab6d352e'}]

#Get_last_timestamp 구현

In [ ]:
def get_last_timestamp(mint_data_transaction,burn_data_transaction,swap_data_transaction):
  #mint_data_transaction은 0일 수가 없다. 
  swap_len = len(swap_data_transaction)
  burn_len = len(burn_data_transaction)
  #Case 1 Swap / Burn 전부 0 인경우
  if(swap_len == 0 and burn_len == 0):
    return last_timestamp = int(mint_data_transaction[-1]['timestamp'])
  #Case 2 Swap_transaction이 0 인경우
  if(swap_len == 0):
    return last_timestamp = int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp']))
  #Case 3 Burn Transaction이 0 인경우
  if(burn_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))
  #Case 4 전부다 있는 경우
  return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))
  

In [18]:
import datetime
date = datetime.datetime.fromtimestamp(1626104040).strftime('%Y-%m-%d %H:%M:%S')


In [19]:
date

'2021-07-12 15:34:00'